In [1]:
# 구글 드라이브 마운트
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# 라이브러리 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# 데이터 불러오기, gameId column 제거
df_all = pd.read_csv('/content/gdrive/My Drive/data/high_diamond_ranked_10min.csv')
df_all = df_all.drop(['gameId'], axis='columns')

df_data = df_all.iloc[:, 1:]
df_label = df_all.iloc[:, 0]

print(df_all.columns)
print(df_data.shape)
print(df_label.shape)

Index(['blueWins', 'blueWardsPlaced', 'blueWardsDestroyed', 'blueFirstBlood',
       'blueKills', 'blueDeaths', 'blueAssists', 'blueEliteMonsters',
       'blueDragons', 'blueHeralds', 'blueTowersDestroyed', 'blueTotalGold',
       'blueAvgLevel', 'blueTotalExperience', 'blueTotalMinionsKilled',
       'blueTotalJungleMinionsKilled', 'blueGoldDiff', 'blueExperienceDiff',
       'blueCSPerMin', 'blueGoldPerMin', 'redWardsPlaced', 'redWardsDestroyed',
       'redFirstBlood', 'redKills', 'redDeaths', 'redAssists',
       'redEliteMonsters', 'redDragons', 'redHeralds', 'redTowersDestroyed',
       'redTotalGold', 'redAvgLevel', 'redTotalExperience',
       'redTotalMinionsKilled', 'redTotalJungleMinionsKilled', 'redGoldDiff',
       'redExperienceDiff', 'redCSPerMin', 'redGoldPerMin'],
      dtype='object')
(9879, 38)
(9879,)


In [4]:
# train set, test set으로 분할
train_size = int(df_all.shape[0] * 0.8)

X_train = df_all.iloc[:train_size, 1:]
X_test = df_all.iloc[train_size:, 1:]

Y_train = df_all.iloc[:train_size, 0]
Y_test = df_all.iloc[train_size:, 0]

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(7903, 38)
(1976, 38)
(7903,)
(1976,)


In [50]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(38, input_dim=38, activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 38)                1482      
_________________________________________________________________
dense_53 (Dense)             (None, 19)                741       
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 20        
Total params: 2,243
Trainable params: 2,243
Non-trainable params: 0
_________________________________________________________________


In [51]:
from keras.callbacks import EarlyStopping

# Early Stopping을 위한 callback
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)
model.fit(X_train, Y_train, epochs=200, batch_size=10, callbacks=[es])

Epoch 1/200
791/791 [==============================] - 1s 1ms/step - loss: 42.9123 - accuracy: 0.6334
Epoch 2/200
791/791 [==============================] - 1s 1ms/step - loss: 18.9414 - accuracy: 0.6476
Epoch 3/200
791/791 [==============================] - 1s 1ms/step - loss: 20.2994 - accuracy: 0.6428
Epoch 4/200
791/791 [==============================] - 1s 1ms/step - loss: 17.1568 - accuracy: 0.6379
Epoch 5/200
791/791 [==============================] - 1s 1ms/step - loss: 14.1507 - accuracy: 0.6417
Epoch 6/200
791/791 [==============================] - 1s 1ms/step - loss: 16.6337 - accuracy: 0.6430
Epoch 7/200
791/791 [==============================] - 1s 1ms/step - loss: 14.2986 - accuracy: 0.6384
Epoch 8/200
791/791 [==============================] - 1s 1ms/step - loss: 13.9058 - accuracy: 0.6475
Epoch 9/200
791/791 [==============================] - 1s 1ms/step - loss: 11.8294 - accuracy: 0.6417
Epoch 10/200
791/791 [==============================] - 1s 1ms/step - loss: 9.1202

In [52]:
_, accuracy = model.evaluate(X_test, Y_test)
print('Accuracy: %.2f' % (accuracy * 100))

62/62 [==============================] - 0s 774us/step - loss: 0.5824 - accuracy: 0.7105
Accuracy: 71.05


In [53]:
# scikit-learn의 패키지를 이용해서 accuracy, precision, recall, f1-score 구하기
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
prediction = model.predict(X_test) > 0.5

acc = 100 * accuracy_score(Y_test, prediction)
pre = 100 * precision_score(Y_test, prediction)
rec = 100 * recall_score(Y_test,prediction)
f1 = 100 * f1_score(Y_test,prediction)

print('accuracy : %.2f%%'%(acc))
print('Precision: %.2f%%\nRecall: %.2f%%\nF1-score: %.2f%% '%(pre, rec, f1))

accuracy : 71.05%
Precision: 66.75%
Recall: 84.84%
F1-score: 74.71% 
